In [1]:
#importando as bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.core.display import HTML

In [2]:
#classe link imagens para HTML
def link_imagem_html(url_imagem):
    return '<img src="'+ url_imagem + '" width="50" >'

In [3]:
#url montada para o primeiro acesso
url = 'https://www.premierleague.com'
tempo = 3

#Efetua a consulta
resposta=requests.get(str(url + '/clubs'), timeout = tempo)
conteudo=resposta.content
site_clube=BeautifulSoup(conteudo, 'html.parser')

In [4]:
#localizando os ítens para alimentar a tabela
pagina_clube = site_clube.find('div', attrs={'class': 'indexSection'})
link_clube = pagina_clube.find_all('li')
url_clube = pagina_clube.find_all('a')

filtra_estadio_clube = site_clube.find_all('div', attrs={'class': 'stadiumName'})

list_info_pag_clubes = []
for x in range(len(url_clube)):
    link_pagina_clube = url + url_clube[x]['href']
    estadio_clube = filtra_estadio_clube[x].text
    
    #Efetua a consulta
    respostasite_clube=requests.get(str(link_pagina_clube), timeout = tempo)
    conteudosite_clube=respostasite_clube.content
    pagina_clube=BeautifulSoup(conteudosite_clube, 'html.parser')

    nome_clube = pagina_clube.find('h1', attrs={'class': 'team js-team'})
    filtra_logo_clube = pagina_clube.find('div', attrs={'class': 'badgeContainer'})
    logo_clube = filtra_logo_clube.find('source')

    #pega dados do atleta
    corta_link_pagina_clube = link_pagina_clube.replace("/overview", '')
    conta_barras = corta_link_pagina_clube.count('/')
    separa_barras = corta_link_pagina_clube.split('/')
    id_clube = separa_barras[4]
    
    list_info_pag_clubes.append([id_clube, nome_clube.text, logo_clube['srcset'], estadio_clube, link_pagina_clube])

infor_clubes = pd.DataFrame(list_info_pag_clubes, columns=['id_clube', 'nome_clube', 'logo_clube', 'estadio_clube', 'url_clube'])

#salvando em csv
infor_clubes.to_csv('backup/equipes_premier_league.csv', index = False)

#carrega tabela com os dados do DataFrame
HTML(infor_clubes.to_html(escape=False, formatters=dict(logo_clube=link_imagem_html)))

,id_clube,nome_clube,logo_clube,estadio_clube,url_clube
0,127,AFC Bournemouth,,Vitality Stadium,https://www.premierleague.com/clubs/127/Bournemouth/overview
1,1,Arsenal,,Emirates Stadium,https://www.premierleague.com/clubs/1/Arsenal/overview
2,2,Aston Villa,,Villa Park,https://www.premierleague.com/clubs/2/Aston-Villa/overview
3,130,Brentford,,Gtech Community Stadium,https://www.premierleague.com/clubs/130/Brentford/overview
4,131,Brighton and Hove Albion,,Amex Stadium,https://www.premierleague.com/clubs/131/Brighton-and-Hove-Albion/overview
5,4,Chelsea,,Stamford Bridge,https://www.premierleague.com/clubs/4/Chelsea/overview
6,6,Crystal Palace,,Selhurst Park,https://www.premierleague.com/clubs/6/Crystal-Palace/overview
7,7,Everton,,Goodison Park,https://www.premierleague.com/clubs/7/Everton/overview
8,34,Fulham,,Craven Cottage,https://www.premierleague.com/clubs/34/Fulham/overview
9,9,Leeds United,,Elland Road,https://www.premierleague.com/clubs/9/Leeds-United/overview


In [5]:
#acessando a página dos clubles
list_atletas = []
for n in infor_clubes.index:
    try:
        url_elenco_clube = infor_clubes['url_clube'][n].replace("overview", 'squad')

        #Efetua a consulta
        resposta_elenco_clube=requests.get(str(url_elenco_clube), timeout = tempo)
        conteudo_elenco_clube=resposta_elenco_clube.content
        site_elenco_clube=BeautifulSoup(conteudo_elenco_clube, 'html.parser')

        #localizando os ítens para alimentar a tabela
        pagina_atleta = site_elenco_clube.find('div', attrs={'data-widget': 'club-squad'})

        link_img_atleta = pagina_atleta.find_all('a', attrs={'class': 'playerOverviewCard active'})
        posicao_atletas = pagina_atleta.find_all('span', attrs={'class': 'position'})

        id_clube_atleta = infor_clubes['id_clube'][n]
        
        #acessando a página dos atletas
        for i in range(len(link_img_atleta)):
            try:
                posicao_atleta = posicao_atletas[i].text
                url_pag_atleta = 'https://www.premierleague.com/'+link_img_atleta[i]['href']

                #Efetua a consulta
                resposta_pag_atleta=requests.get(str(url_pag_atleta), timeout = tempo)
                conteudo_pag_atleta=resposta_pag_atleta.content
                site_pag_atleta=BeautifulSoup(conteudo_pag_atleta, 'html.parser')

                #localizando os ítens para alimentar a tabela
                img_atleta = site_pag_atleta.find('div', attrs={'class': 'imgContainer'})
                infor_atleta = img_atleta.find('img', attrs={'class': 'img'})

                numero_camisa = site_pag_atleta.find('div', attrs={'class': 'number t-colour'})
                nome_atleta = site_pag_atleta.find('div', attrs={'class': 'name t-colour'})
                img_atleta = infor_atleta['src']
                id_atleta = infor_atleta['data-player']

                pais_atleta = site_pag_atleta.find('span', attrs={'class': 'playerCountry'})

                list_atletas.append([numero_camisa.text, nome_atleta.text, posicao_atleta, 
                                    img_atleta.replace("Photo-Missing", id_atleta), pais_atleta.text, id_atleta, id_clube_atleta])
            except:
                pass
    except:
        pass

#carrega os dados dos atletas no DataFrame
infor_atletas = pd.DataFrame(list_atletas, columns=['numero_camisa', 'nome_atleta', 
                                                    'posicao_atleta', 'img_atleta', 'pais_atleta', 'id_atleta', 'id_clube'])

#salvando em csv
infor_atletas.to_csv('backup/atletas_premier_league.csv', index = False)

#carrega tabela com os dados do DataFrame
HTML(infor_atletas.to_html(escape=False, formatters=dict(img_atleta=link_imagem_html)))

,numero_camisa,nome_atleta,posicao_atleta,img_atleta,pais_atleta,id_atleta,id_clube
0,1,Mark Travers,Goalkeeper,,Ireland,p229600,127
1,13,Neto,Goalkeeper,,Brazil,p69752,127
2,45,Cameron Plain,Goalkeeper,,England,p461012,127
3,12,Darren Randolph,Goalkeeper,,Ireland,p32259,127
4,5,Lloyd Kelly,Defender,,England,p235530,127
5,6,Chris Mepham,Defender,,Wales,p223911,127
6,15,Adam Smith,Defender,,England,p54469,127
7,17,Jack Stacey,Defender,,England,p154131,127
8,33,Jordan Zemura,Defender,,Zimbabwe,p447932,127
9,2,Ryan Fredericks,Defender,,England,p81012,127
